In [1]:
pip install ultralytics opencv-python matplotlib


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
from ultralytics import YOLO
import cv2
import datetime
import csv
import os

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt' or your custom-trained model

# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for the default webcam
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Define the directory and file name for the CSV
output_dir = "D:/Logs"  # Specify your desired directory
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
csv_file = os.path.join(output_dir, "object_detection_log.csv")

# Create and initialize the CSV file with headers
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Time", "Object Name", "Object ID"])

# Function to log detections in the CSV file
def log_detection_to_csv(object_name, object_id):
    current_time = datetime.datetime.now()
    date = current_time.strftime("%Y-%m-%d")  # Date part
    time = current_time.strftime("%I:%M:%S %p")  # Time part with AM/PM
    log_entry = [date, time, object_name, object_id]
    print(f"{date} {time} - Object: {object_name}, ID: {object_id}")  # Print to console
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(log_entry)  # Append the log entry to the CSV file

# Initialize detected_objects
detected_objects = set()

# Start real-time object detection
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame from camera.")
        break

    # Perform inference on the current frame
    results = model(frame)

    # Annotate the frame with detected objects
    annotated_frame = results[0].plot()

    # Extract detected objects and IDs
    current_objects = set()
    for obj in results[0].boxes:
        class_id = int(obj.cls)  # Object class ID
        confidence = float(obj.conf)  # Confidence score
        class_name = model.names[class_id]  # Object class name
        current_objects.add((class_name, class_id))

    # Detect changes in objects
    new_objects = current_objects - detected_objects
    removed_objects = detected_objects - current_objects

    # Log new detections to CSV
    for obj in new_objects:
        log_detection_to_csv(obj[0], obj[1])

    # Update the detected objects
    detected_objects = current_objects

    # Display the annotated frame
    cv2.imshow('YOLOv8 Real-Time Detection', annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 480x640 2 persons, 348.6ms
Speed: 15.6ms preprocess, 348.6ms inference, 20.9ms postprocess per image at shape (1, 3, 480, 640)
2025-01-10 12:06:57 PM - Object: person, ID: 0

0: 480x640 1 person, 196.2ms
Speed: 7.4ms preprocess, 196.2ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 177.3ms
Speed: 6.0ms preprocess, 177.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 171.2ms
Speed: 5.0ms preprocess, 171.2ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 176.7ms
Speed: 3.5ms preprocess, 176.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 200.7ms
Speed: 1.3ms preprocess, 200.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 172.2ms
Speed: 3.0ms preprocess, 172.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 163.3ms
Speed: 3.0ms preprocess,

KeyboardInterrupt: 

In [20]:
import pyttsx3
import google.generativeai as genai
import speech_recognition as sr
import time
import sys

# Configure the API key
genai.configure(api_key="AIzaSyAGQYh8WkTJlE1K****************")  # Replace with your actual API key

# Initialize the Gemini model (choose the appropriate model)
model = genai.GenerativeModel("gemini-1.5-flash")  # You can change the model name if needed

# Initialize the TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Set speech rate
engine.setProperty('volume', 1)  # Set volume level

# Change the voice to female (you can change to male or other voices as needed)
def set_voice(gender='female'):
    voices = engine.getProperty('voices')
    if gender == 'female':
        engine.setProperty('voice', voices[1].id)  # Female voice (typically index 1)
    else:
        engine.setProperty('voice', voices[0].id)  # Male voice (typically index 0)

# Initialize the speech recognizer
recognizer = sr.Recognizer()

def process_input(user_input):
    try:
        # Generate a response using the Gemini model
        response = model.generate_content(user_input)
        return response.text
    except Exception as e:
        return f"There was an issue generating a response: {str(e)}"

def talk_to_user(response):
    print(f"Response: {response}")
    engine.say(response)
    engine.runAndWait()

def listen_for_wake_word():
    """Listen for the wake word 'Hey Alpha'."""
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)  # Adjust for background noise
        print("Listening for wake word...")
        while True:
            try:
                # Remove timeout to listen indefinitely
                audio = recognizer.listen(source)  # Listen indefinitely
                query = recognizer.recognize_google(audio)
                if 'hey alpha' in query.lower():  # Wake word detected
                    print("Wake word detected: Hey Alpha. Now listening for command.")
                    talk_to_user("Hi there, this is Alpha model 1.0")
                    return True
            except sr.UnknownValueError:
                pass  # Ignore errors from not recognizing speech
            except sr.RequestError:
                talk_to_user("Sorry, I'm having trouble connecting to the speech service.")
                return False

def listen_to_user():
    """Listen for user input after the wake word 'Hey Alpha' is detected."""
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)  # Adjust for background noise
        print("Listening for command...")
        try:
            audio = recognizer.listen(source, timeout=5)  # Capture audio with a short timeout
            query = recognizer.recognize_google(audio)
            print(f"\nYou said: {query}")
            return query
        except sr.UnknownValueError:
            print("\nSorry, I didn't understand that.")
            talk_to_user("Sorry, I didn't understand that.")
            return None
        except sr.RequestError:
            print("\nSorry, I'm having trouble connecting to the speech service.")
            talk_to_user("Sorry, I'm having trouble connecting to the speech service.")
            return None

def main():
    print("Welcome! Say 'Hey Alpha' to activate.")
    set_voice('female')  # Change voice to female (can use 'male' as argument for male voice)
    while True:
        # Listen for the wake word
        if listen_for_wake_word():
            # Once the wake word is detected, continue listening for commands
            user_input = listen_to_user()
            if user_input:
                if 'exit' in user_input.lower():  # Check if "exit" is in the user input
                    talk_to_user("Goodbye!")  # Say goodbye
                    break  # Exit the loop and stop the program

                # Get the response for the input
                output = process_input(user_input)

                # Speak and display the response
                talk_to_user(output)

if __name__ == "__main__":
    main()


Welcome! Say 'Hey Alpha' to activate.
Listening for wake word...
Wake word detected: Hey Alpha. Now listening for command.
Response: Hi there, this is Alpha model 1.0
Listening for command...

You said: exit
Response: Goodbye!


In [5]:
pip install opencv-python


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [27]:
import cv2
import os
import numpy as np

# Directory to save face images
save_dir = r"D:\myproject\image identification\faceid"
os.makedirs(save_dir, exist_ok=True)

# Initialize OpenCV's Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to capture a photo and save it
def capture_and_save_face():
    name = input("Enter your name: ").strip()
    if not name:
        print("Name cannot be empty.")
        return
    
    cap = cv2.VideoCapture(0)
    print("Press 's' to save the image.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        cv2.imshow("Capture Face", frame)
        
        # Press 's' to save the image
        if cv2.waitKey(1) & 0xFF == ord('s'):
            for (x, y, w, h) in faces:
                face = frame[y:y+h, x:x+w]
                save_path = os.path.join(save_dir, f"{name}.jpg")
                cv2.imwrite(save_path, face)
                print(f"Saved image to {save_path}")
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to recognize faces in real time
def recognize_faces():
    # Load all saved faces and their labels
    known_faces = []
    known_names = []
    
    for filename in os.listdir(save_dir):
        if filename.endswith(".jpg"):
            img_path = os.path.join(save_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            known_faces.append(img)
            known_names.append(os.path.splitext(filename)[0])
    
    cap = cv2.VideoCapture(0)
    print("Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
        
        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (100, 100))  # Resize face for easier comparison
            
            # Compare with known faces
            best_match_name = "Unknown"
            min_diff = float("inf")
            
            for i, known_face in enumerate(known_faces):
                known_face_resized = cv2.resize(known_face, (100, 100))
                diff = np.sum((face - known_face_resized) ** 2)  # Calculate squared difference
                if diff < min_diff:
                    min_diff = diff
                    best_match_name = known_names[i]
            
            # Display result
            color = (0, 255, 0) if best_match_name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, best_match_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        cv2.imshow("Face Recognition", frame)
        
        # Quit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main workflow
print("1. Capture a new face")
print("2. Recognize faces")
choice = input("Enter your choice (1/2): ").strip()

if choice == "1":
    capture_and_save_face()
elif choice == "2":
    recognize_faces()
else:
    print("Invalid choice.")


1. Capture a new face
2. Recognize faces
Press 'q' to quit.


KeyboardInterrupt: 